In [26]:
import pandas as pd
import numpy as np
from numpy import mean
from numpy import std
from sklearn.model_selection import RepeatedKFold
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import accuracy_score
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor

In [239]:
df_spectra=pd.read_csv("spectrum_energy_input_numberOfPeaks.csv",index_col=[0])
#x_data=np.asarray([df_spectra["no_of_max"].values]).transpose()
x_data=np.asarray([df_spectra[["no_of_max","Energy_highest_max_normalized"]].to_numpy()])[0]

y_data=np.asarray([df_spectra[["k6a1","k6a2","k11","k12","k9a1","k9a2"]].to_numpy()])[0]
df_spectra.head(-50)


,main_maximum,all_maxima,no_of_max,k6a1,k6a2,k11,k12,k9a1,k9a2,Energy_highest_max_normalized,Energy_all_max_normalized
0,0.748,[-0.758 -0.69 -0.612 -0.472 -0.406 -0.328 -0....,18,0.25,0.25,0.25,0.25,0.25,0.25,0.874,[0.121 0.155 0.194 0.264 0.297 0.336 0.424 0.4...
1,-0.574,[-0.574 -0.048 0.034 0.596 0.658],5,0.25,0.25,0.25,0.25,0.25,0.50,0.213,[0.213 0.476 0.517 0.798 0.829]
2,0.732,[-0.928 -0.776 -0.708 -0.628 -0.486 -0.422 -0....,19,0.25,0.25,0.25,0.25,0.25,0.75,0.866,[0.036 0.112 0.146 0.186 0.257 0.289 0.33 0.4...
3,0.734,[-0.926 -0.778 -0.712 -0.628 -0.486 -0.422 -0....,19,0.25,0.25,0.25,0.25,0.50,0.25,0.867,[0.037 0.111 0.144 0.186 0.257 0.289 0.33 0.4...
4,-0.560,[-0.56 -0.03 0.046 0.614 0.674],5,0.25,0.25,0.25,0.25,0.50,0.50,0.220,[0.22 0.485 0.523 0.807 0.837]
...,...,...,...,...,...,...,...,...,...,...,...
674,0.736,[-0.774 -0.706 -0.612 -0.488 -0.424 -0.34 -0....,17,0.75,0.75,0.25,0.75,0.75,0.75,0.868,[0.113 0.147 0.194 0.256 0.288 0.33 0.415 0.4...
675,0.730,[-0.922 -0.776 -0.702 -0.624 -0.49 -0.42 -0....,19,0.75,0.75,0.50,0.25,0.25,0.25,0.865,[0.039 0.112 0.149 0.188 0.255 0.29 0.413 0.4...
676,-0.574,[-0.574 -0.046 0.036 0.594 0.66 ],5,0.75,0.75,0.50,0.25,0.25,0.50,0.213,[0.213 0.477 0.518 0.797 0.83 ]
677,0.736,[-0.932 -0.772 -0.736 -0.628 -0.492 -0.424 -0....,20,0.75,0.75,0.50,0.25,0.25,0.75,0.868,[0.034 0.114 0.132 0.186 0.254 0.288 0.331 0.4...


In [276]:
print(y_data.shape)

print(x_data.shape)



(729, 6)
(729, 2)


In [294]:
#always seems to leave some values at zero
def get_model(n_inputs, n_outputs):
    model = Sequential()
    model.add(Dense(20, input_dim=n_inputs, kernel_initializer='he_uniform', activation='relu'))
    #model.add(Dense(25, activation='relu'))
    #model.add(Dense(10, activation='relu'))
    print(n_outputs)
    model.add(Dense(n_outputs, activation='relu'))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

def get_model2(n_inputs, n_outputs):
    model = Sequential()
    model.add(Dense(10, input_dim=n_inputs, kernel_initializer='ones', activation='sigmoid'))
   # model.add(Dense(5, activation='sigmoid'))
   # model.add(Dense(10, activation='sigmoid'))

    #model.add(Dense(10, activation='relu'))
    print(n_outputs)
    model.add(Dense(n_outputs, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam')
    return model

In [313]:
def evaluate_model(X, y):
    results = list()
    n_inputs = X.shape[1]
    print(n_inputs)
    n_outputs = y.shape[1]
    # define evaluation procedure
    cv = RepeatedKFold(n_splits=2, n_repeats=1, random_state=1)
    # enumerate folds
    for train_ix, test_ix in cv.split(X):
        # prepare data
        X_train, X_test = X[train_ix], X[test_ix]
        y_train, y_test = y[train_ix], y[test_ix]
        
        print(X_train[0],y_train[0])
        #print(X_train,y_train)
        # define model
        model = get_model2(n_inputs, n_outputs)
        # fit model
        model.fit(X_train, y_train, verbose=0, epochs=900, batch_size=500)
        # make a prediction on the test set
        yhat = model.predict(X_test)
        # round probabilities to class labels
        yhat = yhat
        # calculate accuracy
        
        rse=model.evaluate(X_test,y_test)
        
        #print(y_test,yhat)
        
        
        # store result
        print('>%.3f' % rse)
        results.append(rse)
    
    #y_predict1=[yhat[i][0] for i in range(len(yhat))]
   # y_predict2=[yhat[i][1] for i in range(len(yhat))]
   #y_test1=[y_test[i][0] for i in range(len(y_test))]
   # y_test2=[y_test[i][1] for i in range(len(y_test))]
    #print(y_test)
    #print(yhat)
    df_test=pd.DataFrame(y_test,columns=["k6a1_test","k6a2_test","k11_test","k12_test","k9a1_test","k9a2_test"])
    df_hat=pd.DataFrame(yhat,columns=["k6a1_hat","k6a2_hat","k11_hat","k12_hat","k9a1_hat","k9a2_hat"])
    

    return results,pd.concat([df_test,df_hat], axis=1)




In [314]:
results,compare = evaluate_model(x_data, y_data)

2
[5.    0.213] [0.25 0.25 0.25 0.25 0.25 0.5 ]
6
12/12 [==============================] - 0s 417us/step - loss: 0.6935
>0.693
[18.     0.874] [0.25 0.25 0.25 0.25 0.25 0.25]
6
12/12 [==============================] - 0s 417us/step - loss: 0.6936
>0.694


In [310]:
print('Accuracy: %.3f (%.3f)' % (mean(results), std(results)))
compare.head(30)

Accuracy: 0.693 (0.000)


,k6a1_test,k6a2_test,k11_test,k12_test,k9a1_test,k9a2_test,k6a1_hat,k6a2_hat,k11_hat,k12_hat,k9a1_hat,k9a2_hat
0,0.25,0.25,0.25,0.25,0.25,0.50,0.497320,0.491620,0.493742,0.502236,0.503891,0.514097
1,0.25,0.25,0.25,0.25,0.75,0.50,0.497320,0.491620,0.493742,0.502236,0.503891,0.514097
2,0.25,0.25,0.25,0.50,0.25,0.50,0.506750,0.496125,0.499132,0.500867,0.498195,0.504482
3,0.25,0.25,0.25,0.50,0.50,0.25,0.496605,0.491537,0.493535,0.502146,0.504101,0.514499
4,0.25,0.25,0.25,0.50,0.50,0.75,0.496605,0.491537,0.493535,0.502146,0.504101,0.514499
5,0.25,0.25,0.25,0.50,0.75,0.25,0.496383,0.491511,0.493468,0.502124,0.504172,0.514626
6,0.25,0.25,0.25,0.75,0.25,0.75,0.494466,0.491291,0.492833,0.502162,0.504974,0.515792
7,0.25,0.25,0.25,0.75,0.50,0.25,0.494466,0.491291,0.492833,0.502162,0.504974,0.515792
8,0.25,0.25,0.25,0.75,0.50,0.50,0.497303,0.491618,0.493737,0.502234,0.503895,0.514107
9,0.25,0.25,0.25,0.75,0.75,0.25,0.494466,0.491291,0.492833,0.502162,0.504974,0.515792
